# 레시피 추천 시스템

https://chan-chance.tistory.com/22

In [32]:
import pandas as pd
import numpy as np
import re
from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from gensim.models import Word2Vec

In [33]:
coupang = pd.read_csv('../../data/coupang_filtering_pre.csv')
coupang.drop('Unnamed: 0', axis=1, inplace=True)

# ckg = pd.read_csv('../../data/ckg_drop.csv')
# ckg.drop('Unnamed: 0', axis=1, inplace=True)

# #결측치 제거
# ckg.dropna(inplace=True)
# ckg.reset_index(inplace=True, drop=True)

## 만개의 레시피에서 조회수 기반 스크랩수 가중치 계산

In [34]:
def weighted_rating(v,m,R,C):
    '''
    Calculate the weighted rating

    ( (각 조회수 / (각 조회수+최소조회수)) * 각 스크랩수 ) + ( (최소조회수/(각 조회수+최소조회수)) * 평균스크랩수 )
    
    Returns: pd.Series
    '''
    return ( (v / (v + m)) * R) + ( (m / (v + m)) * C )

In [35]:
# # calcuate input parameters
# C = np.mean(ckg['스크랩수'])
# m = np.percentile(ckg['조회수'], 70)
# vote_count = ckg[ckg['조회수'] >= m]
# R = ckg['스크랩수']
# v = ckg['조회수']

# ckg['weighted_clipping'] = weighted_rating(v,m,R,C)
# ckg.to_csv('../../data/ckg_weighted.csv') #가중치 추가 저장
# ckg.head()

In [36]:
#가중치까지 계산한 df을 불러옴
ckg = pd.read_csv('../../data/ckg_weighted.csv')
ckg.drop('Unnamed: 0', axis=1, inplace=True)

#word2vec 사용을 위한 데이터 포멧 변경
ckg['요리재료_전처리'] = ckg['요리재료_전처리'].apply(lambda x : x.split(', ')) # 1차원 list 형태
ckg.head()

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
0,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,"[현미, 찹쌀, 호두, 물, 소금]",320.630665
1,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,"[북어포, 찹쌀가루, 간장, 설탕, 물, 다진파, 다진마늘, 참기름, 깨소금, 후춧가루]",289.708247
2,149207,토마토스파게티♥,토마토스파게티,12754,2,36,볶음,일상,가공식품류,면/만두,한번 만들어 두면 이모저모 쓸 용도가 많은 토마토소스를 이용한 토마토 스파게티.,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,"[파스타면, 토마토, 토마토, 양파, 다진마늘, 피망, 올리브유]",216.670718
3,151148,표고버섯탕수,표고버섯탕수,16053,2,194,튀김,손님접대,버섯류,메인반찬,탕수육 먹을때 칼로리때문에 걱정하시는 분들!! 이제는 걱걱걱 정하지 마세요~ 칼로리...,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,"[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일, 녹말가루, 물, 계란 노...",282.469535
4,153040,달콤한 마늘향이 가득해~ 갈릭치킨♥,갈릭치킨,8148,0,38,튀김,손님접대,닭고기,메인반찬,요즘 갈릭이 대세인거 같아요~ 그래서 오늘은 제가 너무 좋아하는 닭과 마늘을 한번에...,[재료] 닭 1마리 [닭양념] 통마늘 3개| 다진마늘 1T| 화이트와인또는우유| 올...,"[닭, 통마늘, 다진마늘, 화이트와인또는우유, 올리브유또는버터, 마늘솔트또는소금, ...",255.980639


In [37]:
coupang[coupang.상세카테고리 == '한식']

,Unnamed: 0.1,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리,상품_전처리
6,6,1,6310053892,13092768757,80353833768,마이셰프 EBS 최요비 산더미 소고기 콩불 2인분,"마이셰프 EBS 최요비 산더미 소고기 콩불 2인분, 810g, 1개",14900,20%,11900,"(100g당 1,469원)",4.5,3015,0,"소고기, 콩나물, 대파, 양파, 깻잎, 소스로 구성되어 있습니다.","소고기,콩나물,대파,양파,깻잎,소스",한식,마이셰프 EBS 최 요 비 산더미 소고기 콩불 2인 분
137,137,2,6988025868,17093723174,84267590270,해물래 멍게비빔밥 재료 키트,"해물래 멍게비빔밥 재료 키트, 270g, 1개",14900,0,14900,"(100g당 5,519원)",3.0,1,0,"멍게, 소스, 양배추, 적양배추, 당근, 고추, 참기름 등으로 구성되어 있습니다.","멍게,소스,양배추,적양배추,당근,고추,참기름",한식,해 물래 멍게 비빔밥 재료 키트
138,138,2,6988025931,17093723512,84267590605,해물래 꼬막비빔밥 재료 키트,"해물래 꼬막비빔밥 재료 키트, 270g, 1개",14900,0,14900,"(100g당 5,519원)",0.0,0,1,"새꼬막, 소스, 양배추, 적양배추, 당근, 고추, 참기름 등으로 구성되어 있습니다.","새꼬막,소스,양배추,적양배추,당근,고추,참기름",한식,해 물래 꼬막 비빔밥 재료 키트
228,228,7,6850319048,16317264703,83509417100,바다자리 매콤한 순살아귀 불고기 + 야채 + 소스 씨키트 2~3인분,"바다자리 매콤한 순살아귀 불고기 + 야채 + 소스 씨키트 2~3인분, 350g, 1개",19150,13%,16650,"(100g당 4,757원)",4.5,285,0,"순살 아귀, 야채 세트(양배추, 양파, 고추, 파, 마늘), 새댁표 고추맛기름, 화...","양배추,양파,고추,파,마늘,화유,하얀설탕",한식,바다 자리 매콤 한 순살 아귀 불고기 + 야채 + 소스 씨 키트 2 ~ 3인 분
234,234,7,6613046365,15003381149,82226223591,마이셰프 불고기 퀘사디아,"마이셰프 불고기 퀘사디아, 374g, 1개",16400,29%,11600,"(100g당 3,102원)",5.0,20,0,"소고기, 또띠아, 양파, 청피망, 표고버섯, 소스 2종, 모짜렐라 치즈로 구성되어 ...","소고기,또띠아,양파,청피망,표고버섯,모짜렐라 치즈",한식,마이셰프 불고기 퀘사디아
268,268,7,6635915899,15150191937,82371721450,살맛나요 간편 갈치조림 (냉동),"살맛나요 간편 갈치조림 (냉동), 320g, 1개",9380,0,9380,"(100g당 2,931원)",4.5,47,0,"갈치, 생선조림용 양념으로 구성되어 있습니다.",갈치,한식,살맛나 요 간편 갈치조림 ( 냉동 )
275,275,7,6675548501,15374719475,82594812838,심플리쿡 남도식 꼬막비빔밥,"심플리쿡 남도식 꼬막비빔밥, 361g, 1개",16900,0,16900,"(100g당 4,681원)",4.5,49,1,"새꼬막살, 채소 키트, 미나리, 꼬막 비빔밥용 소스, 볶음참깨와 흑임자, 참기름 2...","새꼬막살,미나리,볶음참깨,흑임자,고춧가루",한식,심플리쿡 남 도식 꼬막 비빔밥
278,278,7,188130052,537230692,4402737284,프레시지 기사식당 돼지불고기,"프레시지 기사식당 돼지불고기, 501g, 1개",10890,20%,8640,"(100g당 1,725원)",4.5,484,0,"돼지고기, 야채(양배추, 양파, 대파, 양파, 당근), 숯불맛 베이스, 참깨로 구성...","돼지고기,야채,양배추,양파,대파,양파,당근,참깨",한식,프레시지 기사 식당 돼지 불고기
299,299,7,6758563230,15831998258,83043635777,삼삼해물 곰표 쉐이크 잍 해물 김치부침개 (냉동),"삼삼해물 곰표 쉐이크 잍 해물 김치부침개 (냉동), 420g, 1개",7800,0,7800,"(100g당 1,857원)",0.0,0,1,"바삭 파우더, 김치, 오징어채, 반죽 봉투로 구성되어 있습니다.","김치,오징어채",한식,삼 삼 해물 곰표 쉐이크 잍 해물 김치 부침개 ( 냉동 )
306,306,7,6413142502,13762408316,81013061216,푸드어셈블 콩나물 불고기,"푸드어셈블 콩나물 불고기, 963g, 1개",12400,0,12400,"(100g당 1,288원)",0.0,0,1,"콩나물, 돼지고기, 깻잎, 다진 마늘, 양파, 대파, 양념, 참기름으로 구성되어 있...","콩나물,돼지고기,깻잎,다진마늘,양파,대파,양념,참기름",한식,푸드 어 셈 블 콩나물 불고기


In [143]:
a = ckg[ckg.요리명.str.contains('순두부')]
a#[a.요리방법별명 == '찜']

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
47,521163,얼큰 시원한 냉이순두부찌개,냉이순두부찌개,5211,6,7,끓이기,일상,콩/견과류,찌개,보글보글 냉이순두부찌개~ 이거 하나면 다른 반찬 필요없어요.,[재료] 멸치다시마육수 2컵| 냉이 데친것 1컵| 바지락조개 1컵| 계란노른자 1개...,"[멸치다시마육수, 냉이 데친것, 바지락조개, 계란노른자, 애호박, 양파, 대파, 풋...",281.650620
65,556473,순두부찌개,순두부찌개,26757,44,404,끓이기,일상,콩/견과류,찌개,고기가 들어가서 해물을 넣는것과는 또 다른맛이 나용. 더 감칠맛이 좋다랄까용. 적극...,[재료] 순두부 1팩| 돼지고기 1줌| 양파 1/2개| 애호박 1/4개| 물 1컵|...,"[순두부, 돼지고기, 양파, 애호박, 물, 달걀, 청양고추, 홍고추, 대파, 고춧가...",398.708484
170,695402,간단 순두부찌개,순두부찌개,284973,43,1183,끓이기,일상,콩/견과류,찌개,아주 최소의 재료와 양념만으로 간단하게 끓여낸 순두부찌개!,[재료] 순두부 1봉지| 진한 멸치다시마 육수 1/2컵| 달걀 1개| 대파나 쪽파 ...,"[순두부, 진한 멸치다시마 육수, 달걀, 대파나 쪽파, 새우젓, 진간장, 맛술, 다...",1147.212454
318,786019,얼큰하고 시원한 해물 순두부찌개,해물순두부찌개,11000,6,35,끓이기,일상,해물류,찌개,얼큰하고 시원한 해물 순두부찌개 몽글몽글 부드러운 순두부와 쫄깃한 해물에 뜨끈하고...,[재료] 순두부 450그램| 양파 1/2개| 대파 1줄기| 붉은고추 2개| 모둠해물...,"[순두부, 양파, 대파, 붉은고추, 모둠해물, 계란 노른자, 국선생 해물맛, 두반장...",229.188988
700,1051346,순두부 찌개,순두부찌개,67498,31,1154,끓이기,일상,콩/견과류,찌개,밥에 슥슥 비벼 먹으면 꿀맛. 얼큰한 순두부 찌개 하나만 끓여놓아도 김치만 있으면 ...,[재료] 순두부 500g| 바지락 200g| 양파 60g| 애호박 80g| 풋고추 ...,"[순두부, 바지락, 양파, 애호박, 풋고추, 붉은 고추, 대파, 멸치육수, 식용유,...",1026.863297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94853,6952158,해물 순두부찌개 끓이는법 양념장 소스 없어도 돼요,해물순두부찌개,2896,0,35,끓이기,일상,해물류,찌개,순두부찌개 끓이기! 쉬운듯 어려워요 음식점에서 먹는 순두부찌개 맛을 집에서는 내기가...,[재료] 순두부 1팩| 양파 1/2개| 대파 1/2개| 청양고추 2개| 물 360m...,"[순두부, 양파, 대파, 청양고추, 물, 바지락, 칵테일 새우, 오징어, 계란, 간...",325.546296
94985,6952321,먹으면서 빼자! 오트밀 순두부죽,오트밀순두부죽,2358,0,85,끓이기,다이어트,곡류,밥/죽/떡,오늘은 바쁜 아침시간에 간편하게 준비할 수 있는 순두부 오트밀 죽을 소개해드려요. ...,[재료] 누른 오트밀 100ml| 순두부 150g| 애호박 1/4개| 당근 1/4개...,"[누른, 순두부, 애호박, 당근, 치킨스톡, 피쉬소스]",342.908200
94996,6952334,두부의 부드러운 매력 백종원의 매콤한 '순두부찌개',순두부찌개,1808,0,21,끓이기,일상,콩/견과류,찌개,두부의 부드러움 끝판왕 순두부로 맛있는 순부두찌개를 끓여보았습니다~ 백종원님의 레시...,[재료] 순두부 1팩| 간돼지고기 2/5컵| 양파 약 1/3개| 대파 송송| 청양고...,"[순두부, 간돼지고기, 양파 약, 대파, 청양고추, 쪽파, 물, 팽이버섯, 다진마늘...",344.622810
95032,6952381,마라순두부탕~마라탕을 처음 드시는분도 드실수가 있어요,마라순두부탕,2083,0,39,끓이기,초스피드,가공식품류,국/탕,요즘 핫한 마라소스로 순두부를 뜸뿍넣어 탕을 만들어봤어요 초보마라탕입문자들도 무난하...,[재료] 순두부 2봉지| 대파 1/2개| 봄동 6~7장| 돼지고기 150g [양념]...,"[순두부, 대파, 봄동, 돼지고기, 마라 소스, 올리브유, 다진마늘, 소금, 물]",341.304151


# 사전 정의

분리해야하는 세부카테고리 : 탕, 순대, 한식

In [39]:
# 만개의레시피의 카테고리를 기준으로 쿠팡의 세부 카테고리를 분류
ckg_dict = {
    '요리방법별명' : ['끓이기', '굽기', '볶음', '튀김', '기타', '부침', '조림', '찜', '무침', '절임', '비빔', '데치기', '삶기', '회'],
    '요리상황별명' : ['일상', '술안주', '손님접대', '도시락', '야식', '간식', '영양식', '명절', '초스피드','다이어트', '해장', '이유식'],
    '요리재료별명' : ['쌀', '건어물류', '가공식품류', '버섯류', '닭고기', '해물류', '채소류', '육류', '밀가루','콩/견과류', '돼지고기', '달걀/유제품', '소고기', '곡류', '과일류'],
    '요리종류별명' : ['밥/죽/떡', '메인반찬', '면/만두', '양식', '밑반찬', '찌개', '국/탕', '김치/젓갈/장류', '퓨전']
}

#한식의 '밥/죽/떡', '메인반찬' 
#순대의 '밥/죽/떡', '메인반찬' 
## 각각 같은 카테고리로 충돌 > 메인반찬의 카테고리 레시피만 남음 >> 한식의 비빔밥, 순대의 국밥은 아예 결과가 제대로 안 나옴
### >> 쿠팡에서 세부카테고리를 조정하면 어떨지?
coupang_dict = {
    '전골':'국/탕', 
    '생선탕':'국/탕',  #######이것보단 해물류로?
    '찌개/짜글이':'찌개', 
    '한식':'밥/죽/떡,메인반찬,굽기', #######
    '알탕':'국/탕', #######이것보단 해물류로?
    '해물탕':'국/탕', 
    '기타':'null', 
    '국':'국/탕', 
    '씨푸드':'해물류', 
    '구이':'굽기',
    '순대':'밥/죽/떡,메인반찬', #######
    '찜':'찜', 
    '닭':'닭고기', 
    '면':'면/만두', 
    '국수':'면/만두', 
    '스테이크':'굽기,양식', 
    '파스타':'면/만두,양식', 
    '감바스':'양식', 
    '볶음':'볶음', 
    '떡볶이':'밥/죽/떡',
    '마파두부':'볶음', 
    '마라탕':'국/탕', 
    '쌈':'메인반찬', 
    '무침':'무침', 
    '샌드위치':'일상,기타', 
    '족발':'돼지고기', 
    '리조또':'밥/죽/떡', 
    '카레':'일상'
}

In [40]:
# 만개의 레시피에서 사용된 양념 재료 사전
sauce_dict = ['가루','설탕','간장','된장','식초','오일','액젓','소금','물','소주','스프','고추장','고춧가루','사이다','소스','케첩','물엿','올리고당','매실청','식용유','후추','젓갈','즙','정종','와인','맛술']


# word2vec 기반
- tfidf와 비교했을 때 w2v의 코사인이 카테고리 분류하지 않아도 유사한 레시피를 잘 찾아줌
- w2v에서 CBOW와 Skip-Gram 사이의 정확도는 정확한 데이터가 있어야 비교 가능할 듯

In [116]:
##### 요리재료와 쿠팡 구성정보 일치 #####
def get_ingredients_pre(word):
    word_list = ['꼬막','가루','어묵','비엔나 소시지','대파','피망','파프리카','무우','양파','미역','오이','식초','간장','오일','스프','아보카도','물엿','올리고당','후추','젓갈','즙','와인','정종','액젓']
    if ('소고기' in word)or('쇠고기' in word): word = '소고기'
    elif ('건면' in word)or('생면' in word)or('파스타 면' in word)or('파스타면' in word)or('라면 사리' in word)or('우동 사리' in word)or('라면사리' in word)or('우동사리' in word)or('칼국수 면' in word)or('칼국수면' in word)or('스파게티 면' in word)or('스파게티면' in word)or('페투치네' in word)or('우동면' in word)or('우동 면' in word)or('숙면' in word)or('소면' in word)or('쫄면' in word):word = '면'
    elif '고추장' in word: word = '고추장'
    elif ('고추' in word) and ('고추장' not in word): word = '고추'
    elif '새우젓' in word: word = '새우젓'
    elif ('새우' in word)and('새우젓' not in word): word = '새우'
    #elif '명태알 소스' in word: word = '명란'
    elif ('닭' in word)or('수비드 찜닭' in word): word = '닭'
    elif ('계란' in word)or('달걀' in word): word = '계란'
    elif ('소금' in word)or('천일염' in word): word = '소금'
    elif '다진' == word: word = '다진마늘'
    elif ('케첩' in word)or('케찹' in word): word = '케첩'
    elif ('식용유' in word)or('올리브유' in word)or('카놀라유' in word)or('기름' in word): word = '식용유'
    else:
        #단어 리스트에 있는 재료명 통일
        for num in range(len(word_list)):
            if word_list[num] in word: 
                word = word_list[num]
    return word


##### input 값 전처리 후 list에 단어별 저장 #####
def get_product_list(num, product_df):
    # 상품명 전처리
    product = product_df.at[num, '상품_전처리'] #string으로 반환
    product_list = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\(\)]', '', product).split(' ') #특수문자제거(만개의레시피 데이터엔 특수문자가 없어서 인식 불가)
    product_list = list(filter(None, product_list)) #null 제거(안 할 경우 레시피 전체가 선택되어 작동 불가)
    
    #구성정보 전처리
    str_list = product_df.at[num, '구성정보_전처리']
    ingredients_list = str_list.split(',')
    
    #구성정보 요리재료명 통일
    for idx in range(len(ingredients_list)):
        ingredients_list[idx] = get_ingredients_pre(ingredients_list[idx])
    
    #구성정보 요리재료의 중복값 제거   
    result = []
    [result.append(x) for x in ingredients_list if x not in result]
    
    return product_list, result #쿠팡의 상품명, 구성정보_전처리의 요리재료명 반환


##### input 상품의 상세카테고리 추출 #####
def get_product_category(num, product_df):
    product_category = coupang_dict[coupang.at[num, '상세카테고리']].split(',') #리스트 반환
    ckg_key_value = {}
    if product_category == ['null']: #쿠팡의 세부카테고리==기타인 경우, null 반환
        ckg_key_value = {'null': 'null'}
    else:
        for category in product_category:
            for ckg_key in ckg_dict.keys(): #만개의레시피 해당 컬럼의 세부카테고리 리스트 저장
                ckg_value = ckg_dict.get(ckg_key) 
                if category in ckg_value: #만개의레시피 세부카테고리에 쿠팡 세부카테고리가 있으면
                    ckg_key_value[ckg_key] = category # { key=만개의레시피 해당컬럼 : value=해당컬럼의 세부 카테고리}로 저장
    return ckg_key_value


##### input 상품과 동일한 요리명 및 카테고리 의 레시피만 추출 #####
def get_recipe_df(product_list, ckg_key_value):
    #1차필터링
    recipe_df = pd.DataFrame() #추출한 만개의 레시피를 저장할 빈 df 생성
    for word in product_list:
        # 쿠팡상품명과 레시피요리명을 동일하게 변경
        if '보일링' in word: word = '해물찜'
        elif '씨 키트' in word: word = '해물'
        elif '냉동' in word: word = 'NaN' #쿠팡 상품명에 냉동이 있는 경우 제외하기 위한 값
        else: pass
        #해당 쿠팡 상품명 단어와 일치하는 요리명의 index만 저장
        recipe = ckg[ckg['요리명'].str.contains(word)] 
        #위에서 생성한 df와 결합
        recipe_df = pd.concat([recipe_df, recipe])
    #2차 필터링
    recipe_df_filter = pd.DataFrame()
    for ckg_key in ckg_key_value.keys():
        if ckg_key == 'null': 
            #세부카테고리==기타 인 경우, 위의 df 그대로 사용 
            recipe_df_filter = recipe_df
        else:
            #해당하는 컬럼의 일치하는 세부카테고리만 저장
            recipe_df_key = recipe_df[recipe_df[f'{ckg_key}'] == ckg_key_value.get(ckg_key)]
            recipe_df_filter = pd.concat([recipe_df_filter, recipe_df_key])
    #한번에 중복값 제거
    recipe_df_filter.drop_duplicates(['레시피일련번호'], keep=False, inplace=True) 
    
    return recipe_df_filter

##### 추출한 만개의 레시피의 요리재료명 통일 및 word2vec을 위한 이중리스트 생성 #####
def get_recipe_pre(recipe_index, recipe_df):
    recipe_lists = []
    for index in recipe_index:
        man_ingredients = recipe_df.loc[index, '요리재료_전처리']
        # 요리재료가 word2vec을 위해 list형태로 저장
        # 반복문으로 재료명 통일
        recipe_list = []
        for man_ingredient in man_ingredients:
            man_ingredient = get_ingredients_pre(man_ingredient)
            # 추가할 필요가 없는 양념재료 제거
            # 양념 사전와 불일치하는 경우에만 recipe_list에 추가
            if man_ingredient in sauce_dict:
                pass
            else:
                recipe_list.append(man_ingredient)
        # 이중 리스트로 '요리재료_전처리' 전체 반환
        recipe_lists.append(recipe_list)
    return recipe_lists


###### 단어 벡터 평균구하기 ######
def vectors(embedding, embedding_model):
    word_embedding=[]
    dec2vec=None
    count=0
    for word in embedding:
        try : 
            # word에 해당하는 단어를 워드투백에서 찾아 리스트에 붙힘
            # 100차원으로 형성됨
            word_embedding.append(embedding_model.wv[word]) 
            count+=1
        except : continue

    word_embedding2=np.array(word_embedding)
    v=word_embedding2.sum(axis=0)  
    
    if type(v)!=np.ndarray:
        return np.zeros(100)
    
    return(v/count)


###### 최종 알고리즘 ######
def wv_mealkit(sim,mealkit_index,top_n,recipe_df):
    # 코사인유사도를 구한 행렬을 역순으로 정렬 -> 유사도가 높은 순의 인덱스로 정렬됨 
    sim_sorted_ind=sim.argsort()[:,::-1]

    # sim_indexes는 이중리스트 상태이므로 이중리스트를 해제한 후 인덱스를 이용해 해당 내용 추출
    similar_indexes=sim_sorted_ind[mealkit_index,1:(top_n+1)]
    recipe = recipe_df.iloc[similar_indexes]  #.sort_values(by='weighted_clipping',ascending=False)
    return recipe


###### 레시피 only 재료 추출 ######
def get_recipe_only(ingredients_list, sim_mealkit_df):
    recipe_ingredient = []
    for idx in range(len(sim_mealkit_df)):
        value_list = sim_mealkit_df.요리재료_전처리.values[idx]
        
        value_set = [] #중복제거
        [value_set.append(x) for x in value_list if x not in value_set]
        
        for ingredient in ingredients_list:
            # input 구성정보와 요리재료가 일치하면 제외
            if ingredient in value_set:
                value_set.remove(ingredient)
        #레시피에만 있는 요리재료 추가
        recipe_ingredient.append(value_set)
    return recipe_ingredient

## 실행코드

In [239]:
# coupang에서 input 값의 df 추출
num = 99


In [240]:
product_df = coupang.iloc[num:num+1, :]
display(product_df)

# input 상품명 및 구성정보 전처리 후 list로 추출
product_list, ingredients_list = get_product_list(num, product_df)

# try:
# input 상품의 만개의 레시피와 일치하는 카테고리만 추출 {dict}
ckg_key_value = get_product_category(num, product_df)
#print(ckg_key_value)

# input 상품명과 일치하는 레시피만 추출
recipe_df = get_recipe_df(product_list, ckg_key_value)
#display(recipe_df)
if len(recipe_df)==0: #일치하는 레시피가 아예 없는 경우 에러발생
    raise
recipe_index = recipe_df.index.tolist()

# 일치하는 만개의 레시피 요리재료명 통일
word = get_recipe_pre(recipe_index, recipe_df)

# 통일한 요리재료명 업데이트
recipe_df['요리재료_전처리'] = word
#recipe_df.to_csv('../../data/test_ingre.csv')

# word list의 가장 마지막에 input 구성정보 추가
word.append(ingredients_list)

# word2vec 모델 생성
wv_model = Word2Vec(sentences=word, vector_size=100, window=20, min_count=1,  sg=1, batch_words=1000, alpha=0.03)

#레시피 별 벡터 평균 추출
wordvec=[vectors(x, wv_model) for x in word]

# 코사인유사도 계산
w2v_sim=cosine_similarity(wordvec,wordvec)

# 코사인유사도 행렬에서 input 상품의 인덱스 저장
mealkit_index = len(w2v_sim)-1

#input 상품과 유사한 레시피 df 추출
sim_mealkit_df = wv_mealkit(w2v_sim, mealkit_index, 5, recipe_df)
display(sim_mealkit_df) 

#레시피에만 있는 재료 추출
print('ingredients = ', ingredients_list)
recipe_ingredients = get_recipe_only(ingredients_list, sim_mealkit_df)
recipe_ingredients = list(filter(None, recipe_ingredients)) # input 구성정보와 요리재료가 모두 일치(빈리스트)하면 제외 
pprint(recipe_ingredients)

# except: 
#     print("유사한 레시피가 없습니다.")


,Unnamed: 0.1,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리,상품_전처리
99,99,1,7033239068,17364427023,84534354874,면사랑 사골떡만두국 밀키트 2인분 (냉동),"면사랑 사골떡만두국 밀키트 2인분 (냉동), 1개, 551g",9900,0,9900,"(100g당 1,797원)",0.0,0,1,"떡, 두 가지 만두, 옛날 고명 지단, 냉동 대파, 조미김 등이 각 2개씩 총 2인...","떡,냉동 대파",면,면사 랑 사골 떡 만 두국 밀키트 2인 분 ( 냉동 )


,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
76988,6927646,애들이 좋아하는 궁중떡볶이,궁중떡볶이,950,0,24,조림,영양식,소고기,면/만두,고추장을 넣어 만든 평범한 떡볶이가 아닌 간장을 넣은 떡볶이 소고기와 표고버섯의 ...,[주재료] 떡 69g| 소고기 25g| 표고버섯 말린것 2g| 대파 36g [양념]...,"[떡, 소고기, 표고버섯, 대파, 다진마늘]",364.122210
79054,6930566,골목식당 보고 먹고싶어진 자장떡볶이~,자장떡볶이,972,0,22,끓이기,일상,쌀,면/만두,골목식당에 자장떡볶이 만드는거 보고 급 먹고싶어져서 만들어봤어요^^,[재료] 떡 1봉| 어묵 1봉| 돼지고기 다짐육 150g| 양파 1/2개| 대파 1...,"[떡, 어묵, 돼지고기, 양파, 대파]",363.467942
55329,6896362,호박만두 만들기,호박만두,7488,0,42,끓이기,일상,채소류,면/만두,호박만두,[재료] 애호박 6개| 소고기 100g| 불린 표고버섯 6개| 만두피 적당량 [애호...,"[애호박, 소고기, 불린 표고버섯, 만두피, 다진 마늘, 대파, 다진 마늘, 다진 ...",264.287809
62383,6906636,만드는데 단 15분! 파만두!,파만두,10294,0,315,굽기,일상,가공식품류,면/만두,지금당장 맛이~~~ㅆ는 맥주안주를 만들어서 벌컥벌컥하고싶다면? 당장 에어프라이어에 ...,[필수재료] 만두 200g| 대파 1/2대| 식용유 넉넉히 [양념소스재료] 양조간장...,"[만두, 대파, 연겨자]",356.429503
15000,6731030,부추 찐만두 만들기,부추만두,18729,10,233,찜,일상,채소류,면/만두,부추는 각종 영양소가 고루 들어 있는데다 강한 항균 성분까지 지니고 있어 환절기 ...,[재료] 만두피 30장| 갈은 돼지고기 200g| 부추 1/4단| 양배추 2장| 양...,"[만두피, 갈은, 부추, 양배추, 양파, 대파, 청주, 황설탕, 대파]",297.818098


ingredients =  ['떡', '대파']
[['소고기', '표고버섯', '다진마늘'],
 ['어묵', '돼지고기', '양파'],
 ['애호박', '소고기', '불린 표고버섯', '만두피', '다진 마늘', '다진 대 파', '통깨', '사골국물'],
 ['만두', '연겨자'],
 ['만두피', '갈은', '부추', '양배추', '양파', '청주', '황설탕']]


- 22번 : 구성정보 전처리에 안 들어가서 힘든 경우
- 33번 : 생선탕이면 해물류로 바꿔야하나?
- 36번 54번 68번 : 애매
- 40번, 50번 79 : 어묵탕을 따로 빼야할 듯... 시간이 정말정말 오래걸림..
- 44번 : 누룽지를 사라는 뜻..?
- 45번 : 왜 하나만 나오지? 
- 80번 : 돔베고기....ㅠ
- 95번 : 음.........